In [38]:
import numpy as np

def rosenbrock(X,funcParams, doGradient):
  """
  Compute the Rosenbrock function and optionally its gradient.
  
  The Rosenbrock function is defined as:
      f(x, y) = (a - x)^2 + b * (y - x^2)^2
  where 'a' and 'b' are parameters that control the shape of the function.

  Parameters:
      X (array-like): A 2-element array representing the point [x, y] in 2D space
                      where the function and gradient are evaluated.
      funcParams (array-like): A 2-element array containing the parameters [a, b]
                                for the Rosenbrock function.
      doGradient (bool): If True, the function also returns the gradient vector.
                          If False, only the function value is returned.

  Returns:
      f (float): The value of the Rosenbrock function at the given point [x, y].
      gradf (numpy.ndarray or None): The gradient vector [df/dx, df/dy] if doGradient is True.
                                      If doGradient is False, returns None for the gradient.
  """
  x = X[0]
  y = X[1]
  
  a = funcParams[0]
  b = funcParams[1]
  
  x2 = x*x
  
  t1 = a-x
  t2 = y-x2
  
  f = t1**2 + b*t2**2
  
  if doGradient:
    dfdx = - 2*t1 - 4*b*x*t2
    dfdy = 2*b*t2
  
    gradf = np.array([dfdx,dfdy])
  
    return f, gradf
  else:
    return f, None

def line_search(X, S, func, funcParams, maxIter, tol):
    """
    Perform a golden section line search to find the optimal alpha that minimizes
    the function F(X + alpha * S) given X and S
    
    Parameters:
        X (array-like): Current point in the space.
        S (array-like): Search direction.
        tol (float): Tolerance for stopping criterion.
        max_iter (int): Maximum number of iterations.

    Returns:
        alpha_star (float): The optimal step size.
    """
    # Golden ratio
    phi = (1 + np.sqrt(5)) / 2
    inv_phi = 1 / phi

    # Initial interval [a, b]
    a = 0
    b = 1
    # Evaluate points within the interval
    alpha1 = b - inv_phi * (b - a)
    alpha2 = a + inv_phi * (b - a)

    # Compute objective function values at the points
    f1 = func(X + alpha1 * S, funcParams, False)
    f2 = func(X + alpha2 * S, funcParams, False)

    # Iteratively narrow the search interval
    for _ in range(maxIter):
        if abs(b - a) < tol:
            break

        # Compare function values and update the interval
        if f1 < f2:
            b = alpha2
            alpha2 = alpha1
            f2 = f1
            alpha1 = b - inv_phi * (b - a)
            f1 = func(X + alpha1 * S, funcParams, False)
        else:
            a = alpha1
            alpha1 = alpha2
            f1 = f2
            alpha2 = a + inv_phi * (b - a)
            f2 = func(X + alpha2 * S, funcParams, False)

    # Optimal step size is the midpoint of the final interval
    alpha_star = (a + b) / 2
    return alpha_star


def fletcherReevesOpt(X0,func, funcParams, maxLineSearchIters, lineSearchTol, absTol, relTol):
  """
  Perform the Fletcher-Reeves Conjugate Gradient optimization algorithm.

  Parameters:
      X0 (numpy.ndarray): The initial guess point for the optimization, e.g. a 2D vector [x, y].
      func (callable): The objective function to be minimized. Should return 
                        the function value and its gradient at a given point.
      funcParams (numpy.ndarray): Parameters required by the objective function.
      maxLineSearchIters (int): The maximum number of iterations allowed for the line search.
      lineSearchTol (float): Tolerance for the line search stopping criterion.
      absTol (float): Absolute tolerance for convergence of the objective function.
      relTol (float): Relative tolerance for convergence of the objective function.

  Returns:
      X (numpy.ndarray): The optimized point in the parameter space where the 
                          objective function reaches a minimum.
      f (float): The value of the objective function at the optimized point.

  """
  # Step 1: Initialize
  X = X0
  iters = 0
  f, gradf = func(X, funcParams, True)
  a = np.dot(gradf, gradf)
  print("iter= ", iters, "x = ", X[0], "y = ", X[1], "f=", f)

  
  while True:

    # If first time or if Fletcher-Reeves fails, then just select steepest decent
    S = -gradf
    
    # Step 2: Line search to find optimal alpha
    alpha_star = line_search(X, S, func, funcParams, maxLineSearchIters, lineSearchTol)

    # Step 3: Check if alpha_star equals zero
    if np.abs(alpha_star)<=1e-10:
      print("Alpha is zero, algorithm exits.")
      break
    
    # Step 4: Update X
    X = X + alpha_star * S
    
    # Step 5: compute function and gradient at new X
    f, gradf = func(X, funcParams, True)

    while True:
      
      # Compute Fletcher-Reeves conjugate direction update:
      b = np.dot(gradf,gradf)
      beta = b/a
      S = -gradf + beta*S
      a = b
      
      # if slope in the new S direction is greater than 0, break
      if np.dot(S,gradf)>=0:
        break
      
      # line search to find optimal alpha
      alpha_star = line_search(X, S, func, funcParams, maxLineSearchIters, lineSearchTol)
      
      # compute new point and evaluate function
      X_new = X + alpha_star * S
      f_new, gradf_new = func(X_new, funcParams, True)

      # check if convergence criteria are met
      if np.abs(f_new - f)/np.abs(f)<= relTol or np.abs(f_new - f)<=absTol:
        return X, f
      else:
        X = X_new
        f = f_new
        gradf = gradf_new
        iters = iters + 1
        print("iter= ", iters, "x = ", X[0], "y = ", X[1], "f=", f)
    iters = iters + 1
    
  return X, f


# Test Case
def testCGWithRosenBrock():
  X0 = np.array([-4.0,4.0])

  a = 4
  b = 100
  rosenbrockFuncParams = np.array([a,b])
  maxLineSearchIters = 100
  lineSearchTol = 1e-5
  relTol = 1e-12
  absTol = 1e-16

  fletcherReevesOpt(X0,rosenbrock, rosenbrockFuncParams, maxLineSearchIters, lineSearchTol, absTol, relTol)


# Run test cases
testCGWithRosenBrock()
      

      
      

  
  
  

iter=  0 x =  -4.0 y =  4.0 f= 14464.0
iter=  1 x =  2.182397788748313 y =  4.758858698800743 f= 3.3052789261719275
iter=  2 x =  2.20193967876644 y =  4.865560811132463 f= 3.261997340751402
iter=  3 x =  2.420764626888233 y =  5.825488906673381 f= 2.6137866863128587
iter=  4 x =  2.6311213679182024 y =  6.873709815906528 f= 2.114809917165767
iter=  5 x =  2.8310822043435646 y =  7.961585777076442 f= 1.6519593412149989
iter=  6 x =  3.0434164556181056 y =  9.213143432646714 f= 1.1575126901627943
iter=  7 x =  3.323266465920134 y =  11.012816737078122 f= 0.5558325520086083
iter=  8 x =  4.030527327371058 y =  16.246149758939502 f= 0.0010317622278332651
iter=  9 x =  4.0306658472398675 y =  16.24634710608792 f= 0.0009410331310874199
iter=  10 x =  4.030670911298055 y =  16.246346361465456 f= 0.0009408519970061899
iter=  11 x =  4.03067207863791 y =  16.24633981576735 f= 0.0009408266299279954
iter=  12 x =  4.030455179366049 y =  16.24432292192105 f= 0.0009335710734259546
iter=  13 x =  4